# Python basic data storage – towards spreadsheets

If we want to manipulate tablular data, like spreadsheets, in Python, what would be the natural data structures, and what problems does Pandas solve?

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

In [1]:
import numpy as np
import pandas as pd

### Lists

Besides a single variable, a list is perhaps the most basic data container in Python.

- It stores multiple values
- and it's super flexible – you're not confined to just one type of object in a list

In [2]:
ll = ['a', 'b', 'c', 'd', 'e']
ll

['a', 'b', 'c', 'd', 'e']

### List access

- We access individual elements by integer position, starting with `0`, and ending with `length-1`
- we can use the `n:m` "slice" notation to access a sequence (which remains a list)
- notice the returned `n:m` slice starts with `n` but ends with `m-1`
- **knowing the correct integer position can be error prone**
- if you don't know where an element is, searching through the list is slow

In [3]:
print('first element:\t', ll[0])
print('1:3 slice:\t', ll[1:3])
print('last element:\t', ll[-1])

first element:	 a
1:3 slice:	 ['b', 'c']
last element:	 e


### Dictionaries

Knowing the correct integer position to grab an item out of a list can be tricky and error prone, so often it's handy to name things and access them by the name. Python has a "dictionary" for just this purpose.

- Dictionaries contain what are called "**key, value pairs**"
- Keys have some constraints, but numbers and strings are fine (immutable)
- Values can be anything
- Dictionary contents are not guaranteed to be stored in any particular order
- **Lookup is super fast!**

In [4]:
dd = {'aa':1, 'bb':2, 'cc':3, 'dd':4, 'ee':5}
dd

{'aa': 1, 'bb': 2, 'cc': 3, 'dd': 4, 'ee': 5}

### Dictionary access

You access the "values" by putting the "key" in square brackets after the varible name.

In [5]:
print("'aa' key's value:", dd['aa'])
print("'cc' key's value:", dd['cc'])

'aa' key's value: 1
'cc' key's value: 3


---

## Spreadsheets? (Tabular data)

Let's define spreadsheets as:

- Data in a grid of columns and rows
- Within a column, all values have a consistent data type *(string, integer, floating point, boolean...)*
- Not all columns in a spreadsheet need to be the same data type

**If we want to store something like a spreadsheet, how might we think of doing that in Python?**

### Lists of lists

Lists can hold other lists, to create an array of values

In [6]:
lol = [[1,2,3], [4,5,6], [7,8,9]]
lol

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

But we have to access elements by integer again

In [7]:
lol[1][2]

6

And we can only do math easily along one direction – the first-level lists are easiest.

In [8]:
for l in lol:
    print(sum(l))

6
15
24


(or changing the loop to a "list comprehension", for a more "pythonic" expression)

In [9]:
[sum(l) for l in lol]

[6, 15, 24]

### Numpy arrays

For 2D arrays of values we can use Numpy arrays. This is getting us closer to a spreadsheet!

In [10]:
nn = np.array(lol)
nn

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

### Numpy array access

These arrays are built to access by integer position, kind of like lists, but easier than lists in 2 (and higher) dimensions

In [11]:
print('first row and first column value:', nn[0,0])
print('second row and third column value:', nn[1,2])

first row and first column value: 1
second row and third column value: 6


### Numpy math

And with these we can easily do math in any direction. axis=0 is down.

These arrays are especially good for doing things like matrix math (linear algebra) and image processing!

*Note: Numpy arrays are stored more efficiently than Python lists and allow mathematical operations to be vectorized, which results in significantly higher performance than with looping constructs in Python
[[ref]](https://medium.com/@ericvanrees/pandas-series-objects-and-numpy-arrays-15dfe05919d7)*

In [12]:
nn.sum(axis=0)

array([12, 15, 18])

and axis=1 is across

In [13]:
nn.sum(axis=1)

array([ 6, 15, 24])

### But in a Numpy array all the data has to be the same type

If you try to store mixed types, it has to adjust to a "lowest common denominator" data type. Here it reverts to a 21-character unicode string representation "<U21", so the numbers are stored as strings instead of integers, so we can't do math on the numbers.

In [14]:
mixed_array = np.array([[1,2,3],['a','b','c']])
mixed_array

array([['1', '2', '3'],
       ['a', 'b', 'c']], dtype='<U21')

---

## Dictionary of lists or arrays would be a decent idea for a spreadsheet

**This is really starting to look more like a spreadsheet because now we have column names, and the columns don't need to be of the same type!** The lists or arrays allow us to do easy math down the columns.

- dictionary key is the column name
- dictionary value is a list or array holding our rows of values, including Nulls or NaNs

In [15]:
dict_of_arrays = {'name':np.array(['bernice', 'jinyue', 'haim']),
                  'level':np.array([1, 2, 3]), 
                  'grade':np.array([2.9, 3.8, np.nan])}
dict_of_arrays

{'name': array(['bernice', 'jinyue', 'haim'], dtype='<U7'),
 'level': array([1, 2, 3]),
 'grade': array([2.9, 3.8, nan])}

### But math is slightly complicated on a dictionary of lists or arrays

- If we want to do math down all the columns we have to
    - iterate through columns
    - deal with non-numeric types
- across the rows is can't be done directly in this form
- there is nothing to guarantee alignment of the values into rows
    - *if you sort one column, the rest don't get sorted at the same time!*
    - *if you add a new column, you need to be very careful that the rows are in the same order!*
- NaNs (nulls) screw up stats like sum and mean

In [16]:
for k,v in dict_of_arrays.items():
    try:
        print('mean of', k, 'column is', v.mean())
    except TypeError:
        print("can't do mean on", k, "column")

can't do mean on name column
mean of level column is 2.0
mean of grade column is nan


### It's hard to return a certain row

In a dictionary of arrays it's easy to access a single column thorugh the "column name", which is the dictionary key, but with a spreadsheet we often want to access a certain row, based on one of the row values.

**For example, if you want to see all of Jinyue's information, you have to**

- Find the index of the "name" array corresponding to Jinyue
- Grab the value from that index in all arrays and pair it with the column name

In [17]:
ind = np.argwhere(dict_of_arrays['name']=='jinyue')[0,0]
row = [(k, v[ind]) for k,v in dict_of_arrays.items()]
row

[('name', 'jinyue'), ('level', 2), ('grade', 3.8)]

## List of Dictionaries makes row access easier

An alternative spreadsheet storage method with the built-in Python data structures is to store each row as a dictionary of values. That has the advantage of keeping the row information all together, always keyed to the "column name".

In [18]:
list_of_dicts = [{'name': 'bernice', 'level': 1, 'grade': 2.9},
                 {'name': 'jinyue', 'level': 2, 'grade': 3.8},
                 {'name': 'haim', 'level': 3, 'grade': np.nan}]

for l in list_of_dicts:
    if l['name']=='jinyue':
        print(l)

{'name': 'jinyue', 'level': 2, 'grade': 3.8}


### But, finding elements and math down columns is hard

We basically need to re-collect all the values in a certain column and then perform the math operation on it:

In [19]:
math_level = [dd['level'] for dd in list_of_dicts]
print('mean level =', sum(math_level)/len(math_level))

mean level = 2.0


---

## Pandas DataFrame is sort of like a more flexible and efficient dictionary of arrays (or array of dictionaries)

### Pandas Data Structures [[ref]](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html)

- **Series**	1D labeled homogeneously-typed array
    - Container for scalars (numbers), strings, or booleans (True/False)
    - Each Series has an Index (names for all entries) and potentially an overall name
    - Based on a Numpy array [[ref]](https://medium.com/@ericvanrees/pandas-series-objects-and-numpy-arrays-15dfe05919d7)
- **DataFrame**	General 2D labeled, size-mutable tabular structure with potentially heterogeneously-typed columns
    - Container for Series
    - Overall Index, and each Series (column) has a name

Printing a DataFrame in a notebook shows the column names/labels along the top, and the "index", which are the row labels along the left-hand side (here just sequential integers).

In [20]:
df = pd.DataFrame(dict_of_arrays)
df

,name,level,grade
0,bernice,1,2.9
1,jinyue,2,3.8
2,haim,3,NaN


We could have created the DataFrame from the list of dictionaries just as easily

In [21]:
pd.DataFrame(list_of_dicts)

,name,level,grade
0,bernice,1,2.9
1,jinyue,2,3.8
2,haim,3,NaN


### Access is easy on a DataFrame

Columns are each a Series with their own Index, and we can access each element by name so we don't make mistakes with integers

- the Index is like a row number in Excel, but more flexible and powerful
- if you don't specify an Index when you create the DataFrame, Pandas will create an integer Index by default
- **the Index can be other types of identifiers like strings or dates**

In [22]:
df['grade']

0    2.9
1    3.8
2    NaN
Name: grade, dtype: float64

### Math is easy on a DataFrame

- Default is down columns
- Strings are ignored or handled in a logical way
- NaNs/Nulls are ignored rather than leading to a NaN result

In [23]:
df.mean()

level    2.00
grade    3.35
dtype: float64

In [24]:
df.sum()

name     bernicejinyuehaim
level                    6
grade                  6.7
dtype: object

- and we can also do calculations across rows

In [25]:
df.mean(axis=1)

0    1.95
1    2.90
2    3.00
dtype: float64

### We can do things like Transpose

As with a Numpy array, we can do flexible manipulations like transpose

In [26]:
df.T

,0,1,2
name,bernice,jinyue,haim
level,1,2,3
grade,2.9,3.8,NaN


#### or even turn the DataFrame into a dictionary of dictionaries

In [27]:
df.to_dict()

{'name': {0: 'bernice', 1: 'jinyue', 2: 'haim'},
 'level': {0: 1, 1: 2, 2: 3},
 'grade': {0: 2.9, 1: 3.8, 2: nan}}

#### or a Numpy array if we need to

In [28]:
df.to_numpy()

array([['bernice', 1, 2.9],
       ['jinyue', 2, 3.8],
       ['haim', 3, nan]], dtype=object)

#### or a JSON object

In [29]:
df.to_json()

'{"name":{"0":"bernice","1":"jinyue","2":"haim"},"level":{"0":1,"1":2,"2":3},"grade":{"0":2.9,"1":3.8,"2":null}}'

#### or save to a CSV or Excel file

In [30]:
df.to_csv('df_out.csv')

## Operations are vectorized – don't iterate through rows!

When most of us learn programming, one of the most common methods we learn (besides if/then statements) are loops.

If you find yourself **iterating through the elements** (rows or columns) of a DataFrame, there's a good chance you're doing something **much slower and more complicated** than it needs to be! ***I hardly ever need to loop through a DataFrame!***

*Note that if we assign to a column name that doesn't exist, Pandas will just create a new column*

#### String functions on columns

In [31]:
df['NAME'] = df['name'].str.upper()
df

,name,level,grade,NAME
0,bernice,1,2.9,BERNICE
1,jinyue,2,3.8,JINYUE
2,haim,3,NaN,HAIM


#### Math on combinations of columns

In [32]:
df['level_grade'] = df['level']/df['grade']
df

,name,level,grade,NAME,level_grade
0,bernice,1,2.9,BERNICE,0.344828
1,jinyue,2,3.8,JINYUE,0.526316
2,haim,3,NaN,HAIM,NaN


#### Filling nulls with values

In [33]:
df['grade_filled'] = df['grade'].fillna(df['grade'].mean())
df

,name,level,grade,NAME,level_grade,grade_filled
0,bernice,1,2.9,BERNICE,0.344828,2.90
1,jinyue,2,3.8,JINYUE,0.526316,3.80
2,haim,3,NaN,HAIM,NaN,3.35


#### Boolean comparisons between columns

In [34]:
df['grade_gt_level'] = df['grade'] > df['level']
df

,name,level,grade,NAME,level_grade,grade_filled,grade_gt_level
0,bernice,1,2.9,BERNICE,0.344828,2.90,True
1,jinyue,2,3.8,JINYUE,0.526316,3.80,True
2,haim,3,NaN,HAIM,NaN,3.35,False


## Series and DataFrame indices are automatically aligned

Both rows and columns in Pandas are ordered (unlike dictionaries), and that order doesn't need to be numerical or alphabetical in either the column or row indexes.

In [35]:
sd = pd.Series(index=[2,0,1], data=['third','first','second'])
sd

2     third
0     first
1    second
dtype: object

#### Assigning our new Series as a column in our DataFrame matches on the index

*Warning: entries that don't have a matching index value in the DataFrame will be dropped! (There are ways to get around this.)*

In [36]:
df['s'] = sd
df

,name,level,grade,NAME,level_grade,grade_filled,grade_gt_level,s
0,bernice,1,2.9,BERNICE,0.344828,2.90,True,first
1,jinyue,2,3.8,JINYUE,0.526316,3.80,True,second
2,haim,3,NaN,HAIM,NaN,3.35,False,third


## Easy to drop rows with NAs

[.dropna() documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

Many options, including drop any row that has an NA, specify certain columns, or a threshold number of NAs to tolerate.

*Here we're not going to make the change in place, but just see what the DataFrame looks like with the NAs dropped.*

In [37]:
df.dropna()

,name,level,grade,NAME,level_grade,grade_filled,grade_gt_level,s
0,bernice,1,2.9,BERNICE,0.344828,2.9,True,first
1,jinyue,2,3.8,JINYUE,0.526316,3.8,True,second


## Also easy to drop duplicates

[.drop_duplicates() documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

Many options here, too, including subsets of columns to consider and which row instance to keep.

In [38]:
df_dup = pd.DataFrame({'col1':['a','b','c','a','b'],'col2':[1,2,3,1,2]})
df_dup

,col1,col2
0,a,1
1,b,2
2,c,3
3,a,1
4,b,2


In [39]:
df_dup.drop_duplicates()

,col1,col2
0,a,1
1,b,2
2,c,3


---

## Pandas Essential Basic Functionality

https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html

Pandas is well suited for many different kinds of data, but most of the time we use it for **tabular data with heterogeneously-typed columns**, as in an SQL table or Excel spreadsheet

Here are just a few of the things that Pandas does well:

- Easy handling of missing data (represented as NaN)
- Automatic and explicit data alignment
- Powerful, flexible group by functionality to perform split-apply-combine operations on data sets, for both aggregating and transforming data
- Intelligent label-based slicing, fancy indexing, and subsetting of large data sets
- Intuitive merging and joining data sets
- Flexible reshaping and pivoting of data sets
- Robust IO tools for loading data from flat files (CSV and delimited), Excel files, databases, and saving / loading data from the ultrafast HDF5 format
- Time series-specific functionality: date range generation and frequency conversion, moving window statistics, date shifting and lagging, etc